In [1]:
import subprocess
import numpy as np
import glob
import os
import time
import pandas as pd
from pathlib import Path
import shutil 

In [2]:
def make_data_path(window_size: float) -> str:
    return f"new_data/new_obsids_w{window_size:.1f}"



DEFAULT_DATA_PATH = "new_data"  # Folder for downloaded obsids 

WINDOW_SIZES = [0.1, 0.2, 0.4, 0.8]  # The window size to use for the search

VERBOSE = 1  # Level of verbosity for the search functions

FILENAMES = [  # List of filenames to search
    'obsid_lists/obsids_b+10_250107.csv',
    #'obsid_lists/obsids_b-10_250107.csv',
    #'obsid_lists/obsids_b+10_250107_subset.csv'
]



def count_detections_for_obsid(detections_path: str, obsid: str) -> int:
    """
    Count how many lines in detections file belong to this obsid.
    Works even if there is no header; assumes obsid is first token per line.
    """
    p = Path(detections_path)
    if not p.exists():
        return 0
    count = 0
    with p.open() as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            # skip header-like lines if any
            if s.lower().startswith("obs") or s.lower().startswith("obsid"):
                continue
            first = s.split()[0]
            if first == str(obsid):
                count += 1
    return count

def delete_obsid_folder(obsid: str, data_path: str, verbose: int = 0) -> None:
    obs_dir = Path(data_path) / str(obsid)
    if obs_dir.exists():
        if verbose:
            print(f"\tNo detections → deleting {obs_dir}")
        shutil.rmtree(obs_dir, ignore_errors=True)


def read_obsids(filenames: list[str], columns: list[str] = ['Obs ID', 'Public Release Date']) -> pd.DataFrame:
    """
    ## Read obsids from a list of files.

    ### Args:
        filenames `list[str]`: List of filenames to read obsids from.
        columns `list[str]` (optional): Defaults to `['Obs ID', 'Public Release Date']`. Columns to read from the files.

    ### Returns:
        `pd.DataFrame`: DataFrame with the obsids.
    """
    obsids = pd.DataFrame(columns=columns)
    for filename in filenames:
        inter_obsids = pd.read_csv(
            filename, header=0, dtype=str, usecols=columns)
        obsids = pd.concat([obsids, inter_obsids], ignore_index=True)
    return obsids


def download_data(obsid: str, verbose: int = 0, data_path: str = DEFAULT_DATA_PATH):
    """
    ## Download data for a given obsid.

    ### Args:
        obsid `str`: Obsid to download data for.
        verbose `int` (optional): Defaults to `0`. Level of verbosity.
        data_path `str` (optional): Defaults to `DEFAULT_DATA_PATH`. Path where all downloaded data is stored.
    """
    if verbose > 0:
        print(f"\tDownloading")
        start_time = time.perf_counter()

    current_path = os.getcwd()
    os.chdir(f"{data_path}")

    command = f'download_chandra_obsid {obsid} evt2,fov,asol,bpix,msk'
    proc = subprocess.run(command, shell=True)

    command = f'cp {obsid}/primary/* {obsid}/'
    proc = subprocess.run(command, shell=True)

    command = f'gunzip {obsid}/*.gz'
    proc = subprocess.run(command, shell=True)

    os.chdir(f"{current_path}")

    if verbose > 0:
        end_time = time.perf_counter()
        print(f"\tFinished downloading")
        print(f"\tDownload time: {end_time - start_time:.0f} seconds")

def process_data(obsid: str, verbose: int = 0, data_path: str = DEFAULT_DATA_PATH) -> bool:
    """
    ## Process the data for a given obsid.
    """
    if verbose > 0:
        print(f"\tProcessing")
        start_time = time.perf_counter()

    current_path = os.getcwd()

    try:
        os.chdir(f"{data_path}/{obsid}/")
    except FileNotFoundError:
        if verbose > 0:
            print(f"\tNo data found")
        return False

    event_file = glob.glob('*evt2.fits*', recursive=True)
    fov_file = glob.glob('*N*fov1.fits*', recursive=True)

    if not event_file or not fov_file:
        if verbose > 0:
            print("\tMissing event or fov file")
        os.chdir(current_path)
        return False

    # mask 
    command = f'dmcopy "{fov_file[0]}" s3.fov clobber=yes'
    subprocess.run(command, shell=True)

    command = f'dmcopy "{event_file[0]}[sky=region(s3.fov)]" 578_evt2_filtered.fits clobber=yes'
    subprocess.run(command, shell=True)

    # fluximage 
    t = time.perf_counter()
    command = 'fluximage 578_evt2_filtered.fits outroot=s3 bands=broad binsize=2 clobber=yes'
    subprocess.run(command, shell=True)
    print("fluximage:", time.perf_counter() - t, "s")

    # mkpsfmap
    t = time.perf_counter()
    command = 'mkpsfmap s3_broad_thresh.img outfile=s3_psfmap.fits energy=1.4967 ecf=0.393 clobber=yes'
    subprocess.run(command, shell=True)
    print("mkpsfmap:", time.perf_counter() - t, "s")

    # wavdetect for 2 scales
    t = time.perf_counter()
    full_scales = "1 2 4 8 16 32"
    reduced_scales = "1 2 4 8 16"

    print(f"\tRunning wavdetect with full scales: {full_scales}")
    proc = subprocess.run(
        f'wavdetect infile=s3_broad_thresh.img '
        f'psffile=s3_psfmap.fits '
        f'expfile=s3_broad_thresh.expmap '
        f'outfile=s3_expmap_src.fits '
        f'scellfile=s3_expmap_scell.fits '
        f'imagefile=s3_expmap_imgfile.fits '
        f'defnbkgfile=s3_expmap_nbgd.fits '
        f'scales="{full_scales}" sigthresh=1e-6 clobber=yes',
        shell=True, capture_output=True, text=True
    )

    # check for scale-related failure
    if "too big for dataset size" in proc.stderr or "wtransform failed" in proc.stderr:
        print("\t⚠️ wavdetect failed at large scale; retrying with smaller scales.")
        proc = subprocess.run(
            f'wavdetect infile=s3_broad_thresh.img '
            f'psffile=s3_psfmap.fits '
            f'expfile=s3_broad_thresh.expmap '
            f'outfile=s3_expmap_src.fits '
            f'scellfile=s3_expmap_scell.fits '
            f'imagefile=s3_expmap_imgfile.fits '
            f'defnbkgfile=s3_expmap_nbgd.fits '
            f'scales="{reduced_scales}" sigthresh=1e-6 clobber=yes',
            shell=True, capture_output=True, text=True
        )

    print("wavdetect:", time.perf_counter() - t, "s")

    # return to main path 
    os.chdir(current_path)

    if verbose > 0:
        end_time = time.perf_counter()
        print(f"\tFinished processing")
        print(f"\tProcessing time: {end_time - start_time:.0f} seconds")

    return True


def search_data(obsid: str, window_size: float = 20.0, verbose: int = 0, data_path: str = DEFAULT_DATA_PATH):
    if verbose > 0:
        print(f"\tSearching")
        start_time = time.perf_counter()

    current_path = os.getcwd()

    command = f"cp auxiliary/search_algorithm.py {data_path}/{obsid}/"
    subprocess.run(command, shell=True)

    os.chdir(f"{data_path}/{obsid}/")

    t = time.perf_counter()
    command = f"python search_algorithm.py {current_path} {window_size} {verbose}"
    proc = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
    print("search_algorithm.py:", time.perf_counter() - t, "s")

    if verbose > 1:
        print(proc.stdout)

    os.chdir(f"{current_path}")

    if verbose > 0:
        end_time = time.perf_counter()
        print(f'\tFinished searching')
        print(f'\tSearching time: {end_time - start_time:.0f} seconds')

def pipeline(obsid: str, window_size: float = 20.0, verbose: int = 0, data_path: str = DEFAULT_DATA_PATH) -> None:
    """
    ## Pipeline to download, process and search data for a given obsid.
    """
    if verbose > 0:
        print(f'Starting obsid: {obsid}')
        start_time = time.perf_counter()

    # check analysed file 
    analysed_file = f'output/analysed_w{window_size:.1f}_forward.txt'
    try:
        analysed = pd.read_csv(analysed_file, header=0, sep=r"\s+", dtype=str)
        already_analysed = str(obsid) in set(analysed['ObsId'].astype(str).values)

        if verbose > 1:
            print(f"\tChecking analysed file: {analysed_file}")
            print(f"\tFirst few ObsIds: {analysed['ObsId'].head().tolist()}")
    except FileNotFoundError:
        analysed = None
        already_analysed = False

    if already_analysed:
        if verbose > 0:
            print(f'\tObsid {obsid} already searched')
        if verbose > 0:
            end_time = time.perf_counter()
            print(f'Finished obsid: {obsid}')
            print(f'Time: {end_time - start_time:.0f} seconds')
        return  # stop early, nothing else to do

    # download data if not already downloaded
    os.makedirs(data_path, exist_ok=True)
    dirs = os.listdir(f"{data_path}/")
    if obsid in dirs:
        if verbose > 0:
            print(f'\tObsid data already downloaded')
    else:
        download_data(obsid, verbose, data_path)
        if not process_data(obsid, verbose, data_path):
            return  # skip if processing failed

    # count detections before search
    detections_file = f'output/detections_w{window_size:.1f}_forward.txt'
    before = count_detections_for_obsid(detections_file, obsid)

    # run search
    search_data(obsid, window_size, verbose, data_path)

    # count detections after search
    after = count_detections_for_obsid(detections_file, obsid)
    new_hits = after - before

    # cleanup if no detections
    try:
        analysed = pd.read_csv(analysed_file, header=0, sep=r"\s+", dtype=str)
        is_analysed_now = str(obsid) in set(analysed['ObsId'].astype(str).values)
    except Exception:
        is_analysed_now = False

    if is_analysed_now and new_hits == 0:
        delete_obsid_folder(obsid, data_path, verbose=verbose)

    if verbose > 0:
        end_time = time.perf_counter()
        print(f'Finished obsid: {obsid}')
        print(f'Time: {end_time - start_time:.0f} seconds')


def start_search(filenames: list, window_size: float = 20.0, data_path: str = DEFAULT_DATA_PATH, limit_observations: bool = False, verbose: int = 0) -> None:
    """
    ## Start the search for a list of filenames.

    ### Args:
        filenames `list`: List of filenames to search.
        window_size `float` (optional): Defaults to `20.0`. Window size to use for the search.
        data_path `str` (optional): Defaults to `DEFAULT_DATA_PATH`. Path where all downloaded data is stored.
        limit_observations `bool` (optional): Defaults to `False`. Whether to only search observations with an exposure time longer than window_size.
        verbose `int` (optional): Defaults to `0`. Level of verbosity.
    """
    obsids = read_obsids(filenames, ['Obs ID', 'Exposure'])
    obsids['Exposure'] = obsids['Exposure'].astype(float)

    if limit_observations:
        obsids = obsids[obsids['Exposure'] > window_size]

    for i, (index, obsid) in enumerate(obsids.iterrows()):
        print(f"progress: {(i+1)/len(obsids) * 100:.2f}%")
        pipeline(obsid['Obs ID'], window_size, verbose, data_path)


def pipeline_multiwindow(
    obsid: str,
    window_sizes: list,
    verbose: int = 0,
    data_path_base: str = "new_data"
) -> None:
    """
    For running the pipeline for one ObsID across multiple window sizes,
    """

    if verbose > 0:
        print(f"Starting ObsID {obsid} for windows {window_sizes}")

    #  check if all windows already analysed or detected
    all_done = True
    for w in window_sizes:
        analysed_file = f"output/analysed_w{w:.1f}_forward.txt"
        detections_file = f"output/detections_w{w:.1f}_forward.txt"

        in_analysed = False
        in_detections = False

        # check analysed file
        if Path(analysed_file).exists():
            analysed = pd.read_csv(
                analysed_file,
                sep=r"\s+",
                dtype=str,
                header=None,
                names=["ObsId"],
                engine="python"
            )
            if str(obsid) in analysed["ObsId"].astype(str).values:
                in_analysed = True

        # check detections file
        if Path(detections_file).exists():
            if count_detections_for_obsid(detections_file, obsid) > 0:
                in_detections = True

        if not (in_analysed or in_detections):
            all_done = False
            break

    if all_done:
        if verbose > 0:
            print(f"\tObsID {obsid} already analysed/detected for all windows, skipping")
        return

    # download + process if not already present
    obsid_path = Path(data_path_base) / str(obsid)
    if not obsid_path.exists():
        download_data(obsid, verbose, data_path_base)
        if not process_data(obsid, verbose, data_path_base):
            return
    else:
        if verbose > 0:
            print(f"\tObsID {obsid} already downloaded/processed")

    # run searches only for windows not already analysed/detected
    for w in window_sizes:
        analysed_file = f"output/analysed_w{w:.1f}_forward.txt"
        detections_file = f"output/detections_w{w:.1f}_forward.txt"

        in_analysed = False
        in_detections = False

        # check analysed file
        if Path(analysed_file).exists():
            analysed = pd.read_csv(
                analysed_file,
                sep=r"\s+",
                dtype=str,
                header=None,
                names=["ObsId"],
                engine="python"
            )
            if str(obsid) in analysed["ObsId"].astype(str).values:
                in_analysed = True

        # check detections file
        if Path(detections_file).exists():
            if count_detections_for_obsid(detections_file, obsid) > 0:
                in_detections = True

        if in_analysed or in_detections:
            if verbose > 0:
                print(f"\tWindow {w:.1f} ks: already analysed/detected, skipping")
            continue

        before = count_detections_for_obsid(detections_file, obsid)
        search_data(obsid, window_size=w, verbose=verbose, data_path=data_path_base)
        after = count_detections_for_obsid(detections_file, obsid)
        new_hits = after - before

        if verbose > 0:
            print(f"\tWindow {w:.1f} ks: {new_hits} new detections")

    # cleanup if no detections in any window
    total_hits = 0
    for w in window_sizes:
        detections_file = f"output/detections_w{w:.1f}_forward.txt"
        total_hits += count_detections_for_obsid(detections_file, obsid)

    if total_hits == 0:
        delete_obsid_folder(obsid, data_path_base, verbose=verbose)


        
 
if __name__ == "__main__":
    obsids = read_obsids(FILENAMES, ["Obs ID", "Exposure"])
    obsids = obsids["Obs ID"].astype(str).tolist()

    for i, obsid in enumerate(obsids):
        print(f"\n=== {i+1}/{len(obsids)}: Running ObsID {obsid} ===")
        pipeline_multiwindow(obsid, WINDOW_SIZES, verbose=VERBOSE, data_path_base="new_data")

    #pipeline_multiwindow("13825", WINDOW_SIZES, verbose=VERBOSE, data_path_base="new_data")



=== 1/6205: Running ObsID 3411 ===
Starting ObsID 3411 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 3411 already analysed/detected for all windows, skipping

=== 2/6205: Running ObsID 3412 ===
Starting ObsID 3412 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 3412 already analysed/detected for all windows, skipping

=== 3/6205: Running ObsID 3726 ===
Starting ObsID 3726 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 3726 already analysed/detected for all windows, skipping

=== 4/6205: Running ObsID 4418 ===
Starting ObsID 4418 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 4418 already analysed/detected for all windows, skipping

=== 5/6205: Running ObsID 5366 ===
Starting ObsID 5366 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 5366 already analysed/detected for all windows, skipping

=== 6/6205: Running ObsID 6315 ===
Starting ObsID 6315 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 6315 already analysed/detected for all windows, skipping

=== 7/6205: Running ObsID 6316 ===
Starting ObsID 6316 for windows [0.1, 0.2, 0.4

# fluximage (04 November 2021): ERROR For multi-OBI datasets like 906 the obi argument must be set
# mkpsfmap (CIAO 4.16.0): ERROR: Cannot load infile 's3_broad_thresh.img'

# mkpsfmap (CIAO 4.16.0): ERROR: Cannot open input file 's3_broad_thresh.img'



wavdetect: 0.2646882822737098 s
	Finished processing
	Processing time: 1 seconds
	Searching
search_algorithm.py: 0.6064263358712196 s
	Finished searching
	Searching time: 1 seconds
	Window 0.1 ks: 0 new detections
	Searching
search_algorithm.py: 0.5992404660210013 s
	Finished searching
	Searching time: 1 seconds
	Window 0.2 ks: 0 new detections
	Searching
search_algorithm.py: 0.6211747168563306 s
	Finished searching
	Searching time: 1 seconds
	Window 0.4 ks: 0 new detections
	Searching
search_algorithm.py: 0.6059919251129031 s
	Finished searching
	Searching time: 1 seconds
	Window 0.8 ks: 0 new detections
	No detections → deleting new_data/906

=== 3862/6205: Running ObsID 907 ===
Starting ObsID 907 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 907 already analysed/detected for all windows, skipping

=== 3863/6205: Running ObsID 909 ===
Starting ObsID 909 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 909 already analysed/detected for all windows, skipping

=== 3864/6205: Running ObsID 910 ===
Star

# fluximage (04 November 2021): ERROR For multi-OBI datasets like 3182 the obi argument must be set
# mkpsfmap (CIAO 4.16.0): ERROR: Cannot load infile 's3_broad_thresh.img'

# mkpsfmap (CIAO 4.16.0): ERROR: Cannot open input file 's3_broad_thresh.img'



wavdetect: 0.2662795027717948 s
	Finished processing
	Processing time: 1 seconds
	Searching
search_algorithm.py: 0.621043604798615 s
	Finished searching
	Searching time: 1 seconds
	Window 0.1 ks: 0 new detections
	Searching
search_algorithm.py: 0.6068367026746273 s
	Finished searching
	Searching time: 1 seconds
	Window 0.2 ks: 0 new detections
	Searching
search_algorithm.py: 0.6009839680045843 s
	Finished searching
	Searching time: 1 seconds
	Window 0.4 ks: 0 new detections
	Searching
search_algorithm.py: 0.6019404400140047 s
	Finished searching
	Searching time: 1 seconds
	Window 0.8 ks: 0 new detections
	No detections → deleting new_data/3182

=== 3917/6205: Running ObsID 3185 ===
Starting ObsID 3185 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 3185 already analysed/detected for all windows, skipping

=== 3918/6205: Running ObsID 3193 ===
Starting ObsID 3193 for windows [0.1, 0.2, 0.4, 0.8]
	ObsID 3193 already analysed/detected for all windows, skipping

=== 3919/6205: Running ObsID 3194 =

# asphist (CIAO 4.16.0): WARNING: source position is off chip, using extended chip coordinates derived from on-axis postion.
# asphist (CIAO 4.16.0): WARNING: source position is off chip, using extended chip coordinates derived from on-axis postion.
# asphist (CIAO 4.16.0): WARNING: source position is off chip, using extended chip coordinates derived from on-axis postion.
# asphist (CIAO 4.16.0): WARNING: source position is off chip, using extended chip coordinates derived from on-axis postion.
# asphist (CIAO 4.16.0): WARNING: source position is off chip, using extended chip coordinates derived from on-axis postion.


Creating 5 instrument maps for obsid 1112
Creating 5 exposure maps for obsid 1112
Combining 5 exposure maps for obsid 1112
Thresholding data for obsid 1112
Exposure-correcting image for obsid 1112

The following files were created:

 The clipped counts image is:
     s3_broad_thresh.img

 The observation FOV is:
     s3_1112.fov

 The clipped exposure map is:
     s3_broad_thresh.expmap

 The exposure-corrected image is:
     s3_broad_flux.img

fluximage: 13.092194932047278 s
mkpsfmap: 3.5857622832991183 s
	Running wavdetect with full scales: 1 2 4 8 16 32
wavdetect: 20.56621056376025 s
	Finished processing
	Processing time: 38 seconds
	Searching
search_algorithm.py: 1.8516537346877158 s
	Finished searching
	Searching time: 2 seconds
	Window 0.1 ks: 0 new detections
	Window 0.2 ks: already analysed/detected, skipping
	Searching
search_algorithm.py: 1.3172533428296447 s
	Finished searching
	Searching time: 1 seconds
	Window 0.4 ks: 0 new detections
	Searching
search_algorithm.py: 1.2325